# Action1使用Wide&Deep模型对movielens进行评分预测
1、完成代码（20points）     2、使用Wide&Deep工具进行评分预测，代码正确（20points）

## 训练过程

In [1]:
from deepctr.feature_column import SparseFeat,get_feature_names
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
# import warnings
# warnings.filterwarnings("ignore")


data = pd.read_csv("ratings.csv")
sparse_features = ["userId", "movieId", 'timestamp']
target = ['rating']

In [2]:
data

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
1048570,7120,168,5.0,1175543061
1048571,7120,253,4.0,1175542225
1048572,7120,260,5.0,1175542035
1048573,7120,261,4.0,1175543376


In [2]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [4]:
data

,userId,movieId,rating,timestamp
0,0,1,3.5,340880
1,0,28,3.5,340785
2,0,31,3.5,340801
3,0,46,3.5,340790
4,0,49,3.5,340774
...,...,...,...,...
1048570,7119,163,5.0,494540
1048571,7119,247,4.0,494524
1048572,7119,254,5.0,494519
1048573,7119,255,4.0,494545


In [13]:
SparseFeat('userId', data['userId'].nunique())

SparseFeat(name='userId', vocabulary_size=7120, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000002B7000AC280>, embedding_name='userId', group_name='default_group', trainable=True)

In [3]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [20]:
linear_feature_columns

[SparseFeat(name='userId', vocabulary_size=7120, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000002B701D09D00>, embedding_name='userId', group_name='default_group', trainable=True),
 SparseFeat(name='movieId', vocabulary_size=14026, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000002B7026A1970>, embedding_name='movieId', group_name='default_group', trainable=True),
 SparseFeat(name='timestamp', vocabulary_size=822889, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x000002B7026A1D90>, embedding_name='timestamp', group_name='default_group', trainable=True)]

In [22]:
data.userId.values

array([   0,    0,    0, ..., 7119, 7119, 7119], dtype=int64)

In [24]:
train_model_input

{'userId': array([3028, 5661,  855, ..., 6098, 3997,   28], dtype=int64),
 'movieId': array([5903,  330, 2269, ..., 9210,  628,   91], dtype=int64),
 'timestamp': array([794577,  15062, 231068, ..., 525539,  11137,  11402], dtype=int64)}

In [4]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [33]:
len(train['timestamp'].values)

838860

In [20]:
# 10epoch使用WDL进行训练
from deepctr.models import WDL
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2, )

Epoch 1/10
2622/2622 [==============================] - 259s 99ms/step - loss: 0.9409 - mse: 0.9346 - val_loss: 0.7499 - val_mse: 0.7367
Epoch 2/10
2622/2622 [==============================] - 149s 57ms/step - loss: 0.3893 - mse: 0.3698 - val_loss: 0.7917 - val_mse: 0.7653
Epoch 3/10
2622/2622 [==============================] - 116s 44ms/step - loss: 0.4004 - mse: 0.3695 - val_loss: 0.8331 - val_mse: 0.7972
Epoch 4/10
2622/2622 [==============================] - 111s 43ms/step - loss: 0.3007 - mse: 0.2647 - val_loss: 0.8258 - val_mse: 0.7887
Epoch 5/10
2622/2622 [==============================] - 104s 39ms/step - loss: 0.2436 - mse: 0.2075 - val_loss: 0.8251 - val_mse: 0.7886
Epoch 6/10
2622/2622 [==============================] - 102s 39ms/step - loss: 0.2255 - mse: 0.1899 - val_loss: 0.8326 - val_mse: 0.7960
Epoch 7/10
2622/2622 [==============================] - 100s 38ms/step - loss: 0.2109 - mse: 0.1745 - val_loss: 0.8491 - val_mse: 0.8112
Epoch 8/10
2622/2622 [===================

In [23]:
# 1 epoch训练
from deepctr.models import WDL
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )

2622/2622 [==============================] - 117s 45ms/step - loss: 0.9451 - mse: 0.9387 - val_loss: 0.7505 - val_mse: 0.7372


In [25]:
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
pred_ans

array([[3.0677881],
       [2.101383 ],
       [4.28821  ],
       ...,
       [4.1646166],
       [4.200996 ],
       [3.806195 ]], dtype=float32)

In [26]:
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 0.8581375181169975


In [22]:
# 10epoch输出RMSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 0.9059249417032296


## NFM

In [20]:
# 使用NFM进行训练
from deepctr.models import NFM
import time
time1 = time.time()
model = NFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用NFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)
time2 = time.time()
print('运行时间为：',time2-time1)

2622/2622 [==============================] - 155s 59ms/step - loss: 1.0215 - mse: 1.0142 - val_loss: 0.7627 - val_mse: 0.7465
test RMSE 0.8652745229116595
运行时间为： 158.18689227104187


## DeepFM

In [19]:
# 使用deepfm进行训练
from deepctr.models import DeepFM
import time
time1 = time.time()
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用NFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)
time2 = time.time()
print('运行时间为：',time2-time1)

2622/2622 [==============================] - 146s 56ms/step - loss: 0.9358 - mse: 0.9293 - val_loss: 0.7522 - val_mse: 0.7389
test RMSE 0.8623804264940155
运行时间为： 149.58299779891968


## WDL

In [21]:
# 使用WDL进行训练
from deepctr.models import WDL
import time
time1 = time.time()
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用NFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)
time2 = time.time()
print('运行时间为：',time2-time1)

2622/2622 [==============================] - 141s 54ms/step - loss: 0.9379 - mse: 0.9317 - val_loss: 0.7539 - val_mse: 0.7409
test RMSE 0.8628441342444185
运行时间为： 143.85020852088928
